<a href="https://colab.research.google.com/github/ShotaArima/practice_natural_language/blob/main/chap.1-Document%20Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第1章 文書のベクトル化

## janomeによる単語分割

In [1]:
!pip install janome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 74.0 MB/s eta 0:00:00


In [2]:
import janome
from janome.tokenizer import Tokenizer

In [3]:
t = Tokenizer()

In [4]:
for token in t.tokenize('私は秋田犬が大好きです。'):
  print(token)

私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
秋田	名詞,固有名詞,地域,一般,*,*,秋田,アキタ,アキタ
犬	名詞,一般,*,*,*,*,犬,イヌ,イヌ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
大好き	名詞,形容動詞語幹,*,*,*,*,大好き,ダイスキ,ダイスキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。


In [5]:
# wakati=Trueで分かち書きモード
for token in t.tokenize('私は秋田犬が大好きです。', wakati=True):
  print(token, end='/')

私/は/秋田/犬/が/大好き/です/。/

In [6]:
# 半角スペースで分割
for token in t.tokenize('私は秋田犬が大好きです。', wakati=True):
  print(token, end=' ')

私 は 秋田 犬 が 大好き です 。 

In [7]:
# リストに格納
words = [token for token in t.tokenize('私は秋田犬が好きです。', wakati=True)]
print(words)

['私', 'は', '秋田', '犬', 'が', '好き', 'です', '。']


## N-gram

In [15]:
# bi-gram
for i in range (len(words)-1):
    print(f"{i}:{words[i:i+2]}")

0:['私', 'は']
1:['は', '秋田']
2:['秋田', '犬']
3:['犬', 'が']
4:['が', '好き']
5:['好き', 'です']
6:['です', '。']


In [26]:
# n-gram
def get_word_n_grams(sentence, n):
    words = [token for token in t.tokenize(sentence, wakati=True)]
    result = []
    for index in range(len(words)):
        result.append(words[index: index+n])
        if index+n >= len(words):
            return result

In [28]:
input = 'すもももももももものうち'
print(get_word_n_grams(input, 3))

[['すもも', 'も', 'もも'], ['も', 'もも', 'も'], ['もも', 'も', 'もも'], ['も', 'もも', 'の'], ['もも', 'の', 'うち']]


### 文字のN-gram

In [29]:
def get_charactor_n_gram(sentence, n):
    result = []
    for index in range(len(sentence)):
        result.append(sentence[index: index+n])
        if index+n >= len(sentence):
            return result

In [31]:
input2 = '私は秋田犬が大好きだ。'
print(get_charactor_n_gram(input2, 2))

['私は', 'は秋', '秋田', '田犬', '犬が', 'が大', '大好', '好き', 'きだ', 'だ。']


## Bag-of-words

In [32]:
def tokenize(sentence):
    return [token for token in t.tokenize(sentence, wakati=True)]

In [33]:
words1 = '私は秋田犬が大好き。'
words2 = '私は犬が少し苦手。'

print(f"words1:{tokenize(words1)}")
print(f"words2:{tokenize(words2)}")

words1:['私', 'は', '秋田', '犬', 'が', '大好き', '。']
words2:['私', 'は', '犬', 'が', '少し', '苦手', '。']


In [34]:
# 文書ベクトルの作成
def get_bag_of_words(document):
    result_dict = {}
    words = tokenize(document)
    for word in words:
        if word not in result_dict:
            result_dict[word] = 1
        else:
            result_dict[word] += 1
    return result_dict

In [35]:
document1 = '私は秋田犬が大好き。秋田犬は私が大好き。'
document2 = '私は犬が少し苦手。'
print(f"document1:{get_bag_of_words(document1)}")
print(f"document2:{get_bag_of_words(document2)}")

document1:{'私': 2, 'は': 2, '秋田': 2, '犬': 2, 'が': 2, '大好き': 2, '。': 2}
document2:{'私': 1, 'は': 1, '犬': 1, 'が': 1, '少し': 1, '苦手': 1, '。': 1}


In [45]:
# 辞書
def make_dictinary(documents):
    result_dict = {}
    index = 1
    for doc in documents:
        words = tokenize(doc)
        for word in words:
            if word not in result_dict:
                result_dict[word] = index
                index+=1
    return result_dict

In [54]:
documents = [document1, document2]
dictionary = make_dictinary(documents)
print(f"dict:{dict}")

dict:{'私': 1, 'は': 2, '秋田': 3, '犬': 4, 'が': 5, '大好き': 6, '。': 7, '少し': 8, '苦手': 9}


In [57]:
# 用例ベクトルの作成
def make_BOW_vectors(documents, dictionary):
    result_vectors = []
    for doc in documents:
        vec = {}
        words = tokenize(doc)
        for entry in dictionary:
            vec[dictionary[entry]]=0
        for word in words:
            vec[dictionary[word]] += 1
        result_vectors.append(vec)
    return result_vectors

In [63]:
vectors = make_BOW_vectors(documents, dictionary)
print(f"結果:{vectors}")

id_to_word = {v: k for k, v in dictionary.items()}
word_occurrences = []
for occurrence in vectors:
    word_dict = {}
    for id_num, count in occurrence.items():
        word = id_to_word[id_num]
        word_dict[word] = count
    word_occurrences.append(word_dict)
for occurrences in word_occurrences:
    print(f"結果:{occurrences}")

結果:[{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 0, 9: 0}, {1: 1, 2: 1, 3: 0, 4: 1, 5: 1, 6: 0, 7: 1, 8: 1, 9: 1}]
結果:{'私': 2, 'は': 2, '秋田': 2, '犬': 2, 'が': 2, '大好き': 2, '。': 2, '少し': 0, '苦手': 0}
結果:{'私': 1, 'は': 1, '秋田': 0, '犬': 1, 'が': 1, '大好き': 0, '。': 1, '少し': 1, '苦手': 1}
